# Red Neuronal con GRU

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score 
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import json

DATA_DIR = Path('../data/nlp-getting-started')
TRAIN_PATH = DATA_DIR / 'train.csv'
TEST_PATH = DATA_DIR / 'test.csv'
LOCATION_TO_COUNTRY_PATH = Path('../data/location_to_country.json')
RANDOM_SEED = 27

train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

with open(LOCATION_TO_COUNTRY_PATH, 'r', encoding='utf-8') as f:
    location_to_country = json.load(f)

# Categóricas: 'country',  'keyword'. Después las voy a mean encodear.
train_df['country'] = train_df['location'].map(location_to_country).fillna('unknown')
test_df['country'] = test_df['location'].map(location_to_country).fillna('unknown')

train_df['keyword'] = train_df['keyword'].fillna('missing')
test_df['keyword'] = test_df['keyword'].fillna('missing')


categorical_features = ['country', 'keyword']

# Numéricas: 'text_length', 'num_hashtags', 'num_mentions', 'num_uppercase_per_word', 'sentiment_score', 'has_url'

# one hot encoding de 'has_url' a mano
train_df['has_url'] = train_df['text'].fillna('').str.contains(r'http[s]?://', regex=True).astype(int)
test_df['has_url'] = test_df['text'].fillna('').str.contains(r'http[s]?://', regex=True).astype(int)

train_df['text_length'] = train_df['text'].fillna('').str.len()
test_df['text_length'] = test_df['text'].fillna('').str.len()

train_df['num_hashtags'] = train_df['text'].str.count('#')
train_df['num_mentions'] = train_df['text'].str.count('@')

test_df['num_hashtags'] = test_df['text'].str.count('#')
test_df['num_mentions'] = test_df['text'].str.count('@')

def uppercase_per_word(text):
    text = str(text)

    # Palabras que tengan al menos una letra alfabética
    words = [w for w in text.split() if any(ch.isalpha() for ch in w)]
    if not words:
        return 0.0

    # Solo letras alfabéticas, para evitar que cuenten símbolos raros
    uppercase_letters = sum(ch.isupper() for ch in text if ch.isalpha())
    return uppercase_letters / len(words)


train_df['num_uppercase_per_word'] = train_df['text'].apply(uppercase_per_word)
test_df['num_uppercase_per_word']  = test_df['text'].apply(uppercase_per_word)

analyzer = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if pd.isna(text) or text.strip() == '':
        return 0.5
    compound = analyzer.polarity_scores(text)['compound']
    return (compound + 1) / 2

train_df['sentiment_score'] = train_df['text'].apply(get_sentiment)
test_df['sentiment_score'] = test_df['text'].apply(get_sentiment)

numeric_features = ['text_length', 'num_hashtags', 'num_mentions', 'num_uppercase_per_word', 'sentiment_score', 'has_url']

embedding_feature = 'text'

# 1. Separar features y target
X = train_df[numeric_features + categorical_features + [embedding_feature]].copy()
y = train_df['target'].copy()

# 2. Split estratificado train/validation (80/20)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_SEED, stratify=y
)

X_train.head()

,text_length,num_hashtags,num_mentions,num_uppercase_per_word,sentiment_score,has_url,country,keyword,text
2721,87,1,0,1.000000,0.18755,1,unknown,devastated,Obama declares disaster for typhoon-devastated...
2259,132,0,0,0.083333,0.55135,0,unknown,deluged,Businesses are deluged with invzices. Make you...
1815,136,0,0,1.333333,0.27205,1,United Kingdom,crashed,Neil_Eastwood77: I AM A KNOBHEAD!! Bin Laden f...
682,139,0,0,1.666667,0.50000,1,unknown,blazing,Morgan Silver Dollar 1880 S Gem BU DMPL Cameo ...
7216,121,0,2,0.210526,0.78595,0,United States of America,weapons,@danagould @WaynesterAtl I agree with backgrou...


In [2]:
from sklearn.model_selection import KFold

def kfold_target_encoding(train_series, target_series, n_splits=5, random_state=RANDOM_SEED):
    encoded = pd.Series(np.nan, index=train_series.index, dtype=float)
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    global_mean = target_series.mean()

    for tr_idx, val_idx in kf.split(train_series):
        fold_df = pd.DataFrame({
            'feature': train_series.iloc[tr_idx],
            'target': target_series.iloc[tr_idx]
        })
        means = fold_df.groupby('feature')['target'].mean()
        encoded.iloc[val_idx] = train_series.iloc[val_idx].map(means)

    encoded.fillna(global_mean, inplace=True)

    full_df = pd.DataFrame({'feature': train_series, 'target': target_series})
    mapping = full_df.groupby('feature')['target'].mean()

    return encoded, mapping, global_mean

mean_encoded_features = []

for col in ['country', 'keyword']:
    train_encoded, mapping, global_mean = kfold_target_encoding(
        X_train[col], y_train
    )
    new_col = f'{col}_target_mean'
    X_train[new_col] = train_encoded
    X_val[new_col]   = X_val[col].map(mapping).fillna(global_mean)
    test_df[new_col] = test_df[col].map(mapping).fillna(global_mean)
    mean_encoded_features.append(new_col)

# Actualizamos lista de numéricas
numeric_features = numeric_features + mean_encoded_features


Normalizamos todas las features para darle a la Red neuronal un mejor input.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[numeric_features])
X_val_num   = scaler.transform(X_val[numeric_features])
X_test_num  = scaler.transform(test_df[numeric_features])

Preparamos el texto para la GRU (tokenizer + padding)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

text_col = embedding_feature  # 'text'
MAX_WORDS = 10000
MAX_LEN   = 40  # tweets, podés cambiar tras ver histograma de longitudes

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train[text_col].fillna(''))

X_train_seq = pad_sequences(
    tokenizer.texts_to_sequences(X_train[text_col].fillna('')),
    maxlen=MAX_LEN
)
X_val_seq = pad_sequences(
    tokenizer.texts_to_sequences(X_val[text_col].fillna('')),
    maxlen=MAX_LEN
)
X_test_seq = pad_sequences(
    tokenizer.texts_to_sequences(test_df[text_col].fillna('')),
    maxlen=MAX_LEN
)
